In [2]:
import pandas as pd
import os
from datetime import datetime

# Define the Library class
class Library:
    def __init__(self, books_filename):
        self.books_filename = books_filename

        # Check if file exists
        if not os.path.exists(self.books_filename):
            # If the file does not exist, create it with the required structure
            with pd.ExcelWriter(self.books_filename, engine='openpyxl') as writer:
                pd.DataFrame(columns=['SerialNumber', 'BookName', 'CopiesAvailable']).to_excel(writer, sheet_name='Books', index=False)
                pd.DataFrame(columns=['Name', 'BookName', 'IssuedDate', 'ReturnedDate', 'PhoneNumber', 'Session']).to_excel(writer, sheet_name='IssuedBooks', index=False)
                pd.DataFrame(columns=['DonorName', 'BookName', 'DonationDate']).to_excel(writer, sheet_name='Donations', index=False)

        # Attempt to load all sheets and handle potential errors
        try:
            self.books_df = pd.read_excel(self.books_filename, sheet_name='Books', engine='openpyxl')
        except ValueError:
            raise ValueError("Worksheet named 'Books' not found in the Excel file.")
        
        try:
            self.issued_books_df = pd.read_excel(self.books_filename, sheet_name='IssuedBooks', engine='openpyxl')
        except ValueError:
            raise ValueError("Worksheet named 'IssuedBooks' not found in the Excel file.")

        try:
            self.donations_df = pd.read_excel(self.books_filename, sheet_name='Donations', engine='openpyxl')
        except ValueError:
            # If the Donations sheet does not exist, initialize an empty DataFrame
            self.donations_df = pd.DataFrame(columns=['DonorName', 'BookName', 'DonationDate'])

    def displayAvailableBooks(self):
        if self.books_df.empty:
            print("No books available to display.")
        else:
            print(f"\n{len(self.books_df)} AVAILABLE BOOKS ARE:")
            for index, row in self.books_df.iterrows():
                print(f" ♦-- {row['SerialNumber']}: {row['BookName']} (Copies Available: {row['CopiesAvailable']})")
            print("\n")

    def borrowBook(self, name, bookname, phone, session):
        if bookname not in self.books_df['BookName'].values:
            print(f"{bookname} BOOK IS NOT AVAILABLE EITHER TAKEN BY SOMEONE ELSE, WAIT UNTIL IT IS RETURNED.\n")
        else:
            book_info = self.books_df[self.books_df['BookName'] == bookname].iloc[0]
            if book_info['CopiesAvailable'] <= 0:
                print(f"No copies of {bookname} are available right now.\n")
                return
            
            print("BOOK ISSUED: THANK YOU, KEEP IT WITH CARE AND RETURN ON TIME.\n")
            self.books_df.loc[self.books_df['BookName'] == bookname, 'CopiesAvailable'] -= 1  # Decrease the number of available copies
            
            # Record the issuance
            issued_date = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            new_record = pd.DataFrame({
                'Name': [name], 
                'BookName': [bookname],
                'IssuedDate': [issued_date],
                'ReturnedDate': [None],
                'PhoneNumber': [phone],
                'Session': [session]
            })
            self.issued_books_df = pd.concat([self.issued_books_df, new_record], ignore_index=True)
            self._save_to_excel()

    def returnBook(self, bookname, name):
        current_date = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        if not ((self.issued_books_df['Name'] == name) & (self.issued_books_df['BookName'] == bookname)).any():
            print(f"{name} did not borrow {bookname} or wrong book information provided.\n")
        else:
            print("BOOK RETURNED: THANK YOU!\n")
            # Update returned date and add the book back to the library
            self.books_df.loc[self.books_df['BookName'] == bookname, 'CopiesAvailable'] += 1  # Increase the number of available copies
            self.issued_books_df.loc[
                (self.issued_books_df['Name'] == name) & (self.issued_books_df['BookName'] == bookname), 'ReturnedDate'] = current_date
            self._save_to_excel()

    def donateBook(self, bookname, donor_name):
        print("BOOK DONATED: THANK YOU VERY MUCH, HAVE A GREAT DAY AHEAD.\n")
        if bookname in self.books_df['BookName'].values:
            self.books_df.loc[self.books_df['BookName'] == bookname, 'CopiesAvailable'] += 1  # Increase the number of available copies
        else:
            new_book = pd.DataFrame({'SerialNumber': [len(self.books_df)+1], 'BookName': [bookname], 'CopiesAvailable': [1]})
            self.books_df = pd.concat([self.books_df, new_book], ignore_index=True)

        # Record the donation
        donation_date = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        new_donation = pd.DataFrame({
            'DonorName': [donor_name],
            'BookName': [bookname],
            'DonationDate': [donation_date]
        })
        self.donations_df = pd.concat([self.donations_df, new_donation], ignore_index=True)
        self._save_to_excel()

    def _save_to_excel(self):
        # Save changes to the same Excel file
        with pd.ExcelWriter(self.books_filename, engine='openpyxl', mode='w') as writer:
            self.books_df.to_excel(writer, sheet_name='Books', index=False)
            self.issued_books_df.to_excel(writer, sheet_name='IssuedBooks', index=False)
            self.donations_df.to_excel(writer, sheet_name='Donations', index=False)

# Define the Student class
class Student:
    def requestBook(self):
        print("So, you want to borrow a book!")
        self.book = input("Enter the name of the book you want to borrow: ")
        return self.book

    def returnBook(self):
        print("So, you want to return a book!")
        name = input("Enter your name: ")
        self.book = input("Enter the name of the book you want to return: ")
        return self.book, name

    def donateBook(self):
        print("Okay! You want to donate a book!")
        self.book = input("Enter the name of the book you want to donate: ")
        self.donor_name = input("Enter your name: ")
        return self.book, self.donor_name

# Main code execution starts here
if __name__ == "__main__":
    try:
        # Initialize the library with the new Excel file path
        eee_seminar_library = Library('D:\\Pyy.xlsx')
        student = Student()

        # Display the welcome message
        print("\t\t\t\t\t\t\t♦♦♦♦♦♦♦ WELCOME TO THE EEE SEMINAR LIBRARY ♦♦♦♦♦♦♦\n")
        print("""CHOOSE WHAT YOU WANT TO DO:-\n1. Listing all books\n2. Borrow books\n3. Return books\n4. Donate books\n5. Track books\n6. Exit the library\n""")

        while True:
            try:
                usr_response = input("Enter your choice: ").strip()
                
                # Validate input to check if it is a digit
                if not usr_response.isdigit():
                    print(f"'{usr_response}' is not a valid choice. Please enter a number between 1 and 6.")
                    continue
                
                usr_response = int(usr_response)
                
                if usr_response == 1:
                    eee_seminar_library.displayAvailableBooks()
                elif usr_response == 2:
                    name = input("Enter your name: ")
                    bookname = student.requestBook()
                    phone = input("Enter your phone number: ")
                    session = input("Enter your session: ")
                    eee_seminar_library.borrowBook(name, bookname, phone, session)
                elif usr_response == 3:
                    bookname, name = student.returnBook()
                    eee_seminar_library.returnBook(bookname, name)
                elif usr_response == 4:
                    bookname, donor_name = student.donateBook()
                    eee_seminar_library.donateBook(bookname, donor_name)
                elif usr_response == 5:
                    if not eee_seminar_library.issued_books_df.empty:
                        for index, row in eee_seminar_library.issued_books_df.iterrows():
                            print(f"{row['BookName']} book is taken/issued by {row['Name']}. Issued on: {row['IssuedDate']}, Returned on: {row['ReturnedDate']}, Phone: {row['PhoneNumber']}, Session: {row['Session']}.")
                    else:
                        print("NO BOOKS ARE ISSUED!.\n")
                elif usr_response == 6:
                    print("THANK YOU! \n")
                    break
                else:
                    print("INVALID INPUT! Please enter a number between 1 and 6.\n")
            except Exception as e:
                print(f"An error occurred: {e} ---> INVALID INPUT! \n")
    except Exception as e:
        print(f"An error occurred: {e}")



							♦♦♦♦♦♦♦ WELCOME TO THE EEE SEMINAR LIBRARY ♦♦♦♦♦♦♦

CHOOSE WHAT YOU WANT TO DO:-
1. Listing all books
2. Borrow books
3. Return books
4. Donate books
5. Track books
6. Exit the library



Enter your choice:  1



16 AVAILABLE BOOKS ARE:
 ♦-- 1: Economics (Copies Available: 9)
 ♦-- 2: Circuit (Copies Available: 20)
 ♦-- 3: Circuit_1 (Copies Available: 12)
 ♦-- 4: Electronics_2 (Copies Available: 12)
 ♦-- 5: PS_1 (Copies Available: 12)
 ♦-- 6: PS_2 (Copies Available: 12)
 ♦-- 7: HV (Copies Available: 12)
 ♦-- 8: Machine_1 (Copies Available: 12)
 ♦-- 9: Machine_2 (Copies Available: 12)
 ♦-- 10: Machine_3 (Copies Available: 12)
 ♦-- 11: PLC (Copies Available: 12)
 ♦-- 12: Microprocessor (Copies Available: 12)
 ♦-- 13: Electronics (Copies Available: 12)
 ♦-- 17: CDDG (Copies Available: 12)
 ♦-- 15: Micro Electronics (Copies Available: 12)
 ♦-- 16: Mathe (Copies Available: 12)




Enter your choice:  6


THANK YOU! 

